# 🤖 MCP 서버 LLM 통합 테스트

이 노트북은 APIM을 통해 배포된 MCP 서버를 **Azure OpenAI**와 연동하여 테스트합니다.

## 📋 개요

### 아키텍처

```
┌─────────────┐     ┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│   사용자    │────▶│    LLM     │────▶│    APIM    │────▶│  MCP 서버   │
│   (Prompt)  │     │  (GPT-5)    │     │  (Gateway)  │     │  (Functions)│
└─────────────┘     └─────────────┘     └─────────────┘     └─────────────┘
                          │                    │
                          │ Function Call      │ Tool Execute
                          ▼                    ▼
                    ┌─────────────────────────────────┐
                    │        Tool Result              │
                    └─────────────────────────────────┘
```

### 학습 내용

| 항목 | 설명 |
|------|------|
| **MCP 클라이언트** | APIM을 통해 MCP 서버와 통신하는 Python 클라이언트 구현 |
| **도구 형식 변환** | MCP 도구 스키마 → OpenAI Function 형식 변환 |
| **LLM 연동** | Azure OpenAI / OpenAI API의 Function Calling 기능 활용 |
| **에이전트 구현** | 대화 기록을 유지하는 MCP 에이전트 |

### 목차

1. [환경 설정](#1) - 패키지 설치 및 연결 설정
2. [MCP 서버 연결 테스트](#2) - MCP 서버 동작 확인
3. [Azure OpenAI와 MCP 연동](#3) - Function Calling으로 도구 연동
4. [대화형 MCP 에이전트](#4) - 대화 기록 유지 에이전트

### 사전 요구사항

- ✅ `apim-mcp.ipynb` 워크샵 완료 (MCP 서버 배포됨)
- ✅ Azure OpenAI 또는 OpenAI API 키
- ✅ Azure OpenAI 리소스 및 API 키

---
<a id='1'></a>
# 1️⃣ 환경 설정

MCP 서버와 LLM을 연동하기 위한 환경을 설정합니다.

### 1.1 패키지 설치

LLM 연동에 필요한 Python 패키지를 설치합니다:

| 패키지 | 용도 |
|--------|------|
| `openai` | Azure OpenAI API 클라이언트 |
| `requests` | MCP 서버 HTTP 요청 |
| `python-dotenv` | `.env` 파일에서 환경 변수 로드 |

In [15]:
# 필요한 패키지 설치
%pip install openai requests python-dotenv --quiet

Note: you may need to restart the kernel to use updated packages.


### 1.2 라이브러리 임포트

필요한 라이브러리를 임포트하고 `.env` 파일에서 환경 변수를 로드합니다.

**임포트되는 모듈:**
- `os`: 환경 변수 접근
- `json`: JSON 파싱 및 직렬화
- `requests`: HTTP 요청
- `AzureOpenAI`: Azure OpenAI 클라이언트
- `load_dotenv`: 환경 변수 로드

In [16]:
import os
import json
import requests
from openai import AzureOpenAI
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

print("✅ 패키지 로드 완료")

✅ 패키지 로드 완료


### 1.3 MCP 서버 설정

APIM을 통해 배포된 MCP 서버에 연결하기 위한 설정입니다.

#### 설정 로드 방식

이 노트북은 `apim-mcp.ipynb`에서 저장한 `workshop_config.json` 파일을 자동으로 읽어옵니다.

```
apim-mcp.ipynb                      mcp-llm-test.ipynb
     │                                     │
     │ WORKSHOP_CONFIG                     │
     │        │                            │
     ▼        ▼                            ▼
 workshop_config.json  ──────────▶  WORKSHOP_CONFIG
     (JSON 파일)                    (자동 로드)
```

#### 사전 요구사항

1. `apim-mcp.ipynb` 실행 완료
2. 섹션 6.4 실행하여 `workshop_config.json` 파일 저장

> ⚠️ **주의**: `workshop_config.json` 파일이 없으면 환경 변수에서 읽어옵니다.

In [ ]:
# ============================================================
# apim-mcp.ipynb에서 저장한 설정 파일 로드
# ============================================================

CONFIG_FILE = "workshop_config.json"
WORKSHOP_CONFIG = {}

# 설정 파일 로드 시도
if os.path.exists(CONFIG_FILE):
    with open(CONFIG_FILE, 'r', encoding='utf-8') as f:
        WORKSHOP_CONFIG = json.load(f)
    print(f"✅ 설정 파일 로드 완료: {CONFIG_FILE}")
else:
    print(f"⚠️ 설정 파일 없음: {CONFIG_FILE}")
    print("   apim-mcp.ipynb의 섹션 6.4를 먼저 실행하세요.")
    print("   환경 변수에서 값을 읽어옵니다.")

# ============================================================
# MCP 서버 설정 (WORKSHOP_CONFIG에서 가져오기)
# ============================================================

# APIM 게이트웨이 URL
apim_gateway_url = WORKSHOP_CONFIG.get('apim_gateway_url') or os.getenv("APIM_GATEWAY_URL", "")

# APIM Subscription Key
subscription_key = WORKSHOP_CONFIG.get('apim_subscription_key') or os.getenv("APIM_SUBSCRIPTION_KEY", "")

# MCP 서버 엔드포인트
MCP_BASE_URL = f"{apim_gateway_url}/mcp" if apim_gateway_url else ""

# ============================================================
# Entra ID 설정 (WORKSHOP_CONFIG에서 가져오기)
# ============================================================

# Tenant ID
tenant_id = WORKSHOP_CONFIG.get('tenant_id') or os.getenv("AZURE_TENANT_ID", "")

# API App ID (MCP-API)
api_app_id = WORKSHOP_CONFIG.get('entra_api_app_id') or os.getenv("ENTRA_API_APP_ID", "")

# Client App ID (MCP-Client)
client_app_id = WORKSHOP_CONFIG.get('entra_client_app_id') or os.getenv("ENTRA_CLIENT_APP_ID", "")

# Client Secret
client_secret = WORKSHOP_CONFIG.get('entra_client_secret') or os.getenv("ENTRA_CLIENT_SECRET", "")

# JWT 토큰 (직접 설정하거나 아래에서 자동 획득)
JWT_TOKEN = os.getenv("JWT_TOKEN", "")

def get_jwt_token() -> str:
    """
    Entra ID OAuth 2.0 Client Credentials Flow로 JWT 토큰 획득
    """
    global JWT_TOKEN
    
    # 이미 토큰이 있으면 반환
    if JWT_TOKEN:
        return JWT_TOKEN
    
    # 필수 값 확인
    if not all([tenant_id, client_app_id, client_secret, api_app_id]):
        print("⚠️ Entra ID 설정이 없습니다. JWT 토큰 없이 진행합니다.")
        return ""
    
    # 토큰 요청
    token_url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token"
    token_data = {
        "client_id": client_app_id,
        "client_secret": client_secret,
        "scope": f"api://{api_app_id}/.default",
        "grant_type": "client_credentials"
    }
    
    try:
        response = requests.post(token_url, data=token_data, timeout=10)
        
        if response.status_code == 200:
            token_response = response.json()
            JWT_TOKEN = token_response.get("access_token", "")
            expires_in = token_response.get("expires_in", 0)
            print(f"✅ JWT 토큰 획득 성공! (만료: {expires_in}초)")
            return JWT_TOKEN
        else:
            print(f"❌ 토큰 발급 실패: {response.status_code}")
            print(f"   {response.text[:200]}")
            return ""
    except Exception as e:
        print(f"❌ 토큰 요청 오류: {e}")
        return ""

# JWT 토큰 자동 획득 시도
if not JWT_TOKEN and all([tenant_id, client_app_id, client_secret, api_app_id]):
    print("🔐 Entra ID에서 JWT 토큰 획득 중...")
    JWT_TOKEN = get_jwt_token()

print()
print("📌 MCP 서버 설정:")
print(f"   🌐 apim_gateway_url: {apim_gateway_url if apim_gateway_url else '❌ 설정 필요'}")
print(f"   🔗 MCP Base URL: {MCP_BASE_URL if MCP_BASE_URL else '❌ 설정 필요'}")
print(f"   🔑 subscription_key: {'✅ 설정됨 (' + subscription_key[:5] + '...)' if subscription_key else '❌ 설정 필요'}")
print()
print("📌 Entra ID 설정:")
print(f"   🏢 tenant_id: {'✅ ' + tenant_id[:5] + '...' if tenant_id else '⚪ 미설정'}")
print(f"   📱 client_app_id: {'✅ ' + client_app_id[:5] + '...' if client_app_id else '⚪ 미설정'}")
print(f"   🔐 client_secret: {'✅ ' + client_secret[:5] + '***...' if client_secret else '⚪ 미설정'}")
print(f"   🎯 api_app_id: {'✅ ' + api_app_id[:5] + '...' if api_app_id else '⚪ 미설정'}")
print(f"   🎫 JWT_TOKEN: {'✅ 획득됨 (' + JWT_TOKEN[:5] + '...)' if JWT_TOKEN else '⚪ 없음'}")

🔐 Entra ID에서 JWT 토큰 획득 중...
❌ 토큰 발급 실패: 400
   {"error":"unauthorized_client","error_description":"AADSTS700016: Application with identifier 'your-client-app-id' was not found in the directory 'Contoso'. This can happen if the application has not 

📌 MCP 서버 설정 (apim-mcp.ipynb WORKSHOP_CONFIG와 동일):
   🌐 apim_gateway_url: https://mcp-dev-apim-i6ht.azure-api.net
   🔗 MCP Base URL: https://mcp-dev-apim-i6ht.azure-api.net/mcp
   🔑 subscription_key: ✅ 설정됨 (6545720d...)

📌 Entra ID 설정 (apim-mcp.ipynb WORKSHOP_CONFIG와 동일):
   🏢 tenant_id: ✅ 설정됨 (38bd4dc5...)
   📱 client_app_id: ✅ 설정됨 (your-cli...)
   🔐 client_secret: ✅ 설정됨 (your***...)
   🎯 api_app_id: ✅ 설정됨 (your-api...)
   🎫 JWT_TOKEN: ⚪ 없음


### 1.4 Azure OpenAI 설정

Azure OpenAI 연결 정보를 설정합니다.

#### 필요한 환경 변수

| 환경 변수 | 설명 | 예시 |
|-----------|------|------|
| `AZURE_OPENAI_ENDPOINT` | Azure OpenAI 리소스 URL | `https://your-resource.openai.azure.com` |
| `AZURE_OPENAI_API_KEY` | Azure OpenAI API 키 | Azure Portal에서 확인 |
| `AZURE_OPENAI_DEPLOYMENT` | 배포된 모델 이름 | `gpt-5` |

> 💡 **Tip**: `.env` 파일에 환경 변수를 설정하면 코드에 민감한 정보를 직접 입력하지 않아도 됩니다.

```
AZURE_OPENAI_ENDPOINT=https://your-resource.openai.azure.com
AZURE_OPENAI_API_KEY=your-key
AZURE_OPENAI_DEPLOYMENT=gpt-5
```

In [ ]:
# ============================================================
# Azure OpenAI 설정
# ============================================================

# Azure OpenAI 연결 정보
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT", "https://your-resource.openai.azure.com")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY", "your-azure-openai-key")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT", "gpt-5")  # 배포 이름
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-12-01-preview")

print(f"📌 Azure OpenAI 설정:")
print(f"   🌐 Endpoint: {AZURE_OPENAI_ENDPOINT}")
print(f"   📦 Deployment: {AZURE_OPENAI_DEPLOYMENT}")
print(f"   🔑 API Key: {'설정됨' if AZURE_OPENAI_API_KEY != 'your-azure-openai-key' else '❌ 설정 필요'}")

---
<a id='2'></a>
# 2️⃣ MCP 서버 연결 테스트

LLM 연동 전에 MCP 서버가 정상적으로 작동하는지 확인합니다.

### 테스트 항목

| 테스트 | 엔드포인트 | 설명 |
|--------|------------|------|
| Health Check | `GET /mcp/health` | 서버 상태 확인 |
| List Tools | `GET /mcp/tools` | 사용 가능한 도구 목록 |
| Call Tool | `POST /mcp/tools` | 도구 실행 테스트 |

### 2.1 MCP 클라이언트 클래스

MCP 서버와 통신하기 위한 Python 클라이언트 클래스를 정의합니다.

**클래스 구조:**

```python
class MCPClient:
    ├── __init__(base_url, subscription_key, jwt_token)  # 초기화
    ├── health_check() -> dict                           # 서버 상태 확인
    ├── list_tools() -> list                             # 도구 목록 조회
    ├── call_tool(name, arguments) -> dict               # 도구 실행
    └── send_message(method, params) -> dict             # JSON-RPC 메시지 전송
```

**HTTP 헤더:**
- `Content-Type: application/json`
- `Ocp-Apim-Subscription-Key`: APIM 인증
- `Authorization: Bearer {token}`: JWT 인증 (선택)

In [ ]:
# MCP 서버 헬퍼 클래스
class MCPClient:
    """MCP 서버와 통신하는 클라이언트"""
    
    def __init__(self, base_url: str, subscription_key: str, jwt_token: str = ""):
        self.base_url = base_url
        self.headers = {
            "Content-Type": "application/json",
            "Ocp-Apim-Subscription-Key": subscription_key
        }
        if jwt_token:
            self.headers["Authorization"] = f"Bearer {jwt_token}"
    
    def health_check(self) -> dict:
        """서버 상태 확인"""
        response = requests.get(f"{self.base_url}/health", headers=self.headers)
        response.raise_for_status()
        return response.json()
    
    def list_tools(self) -> list:
        """사용 가능한 도구 목록 조회"""
        response = requests.get(f"{self.base_url}/tools", headers=self.headers)
        response.raise_for_status()
        return response.json().get("tools", [])
    
    def call_tool(self, name: str, arguments: dict) -> dict:
        """도구 실행"""
        payload = {"name": name, "arguments": arguments}
        response = requests.post(f"{self.base_url}/tools", headers=self.headers, json=payload)
        response.raise_for_status()
        return response.json()
    
    def send_message(self, method: str, params: dict = None) -> dict:
        """JSON-RPC 스타일 메시지 전송"""
        payload = {
            "jsonrpc": "2.0",
            "method": method,
            "params": params or {},
            "id": 1
        }
        response = requests.post(f"{self.base_url}/messages", headers=self.headers, json=payload)
        response.raise_for_status()
        return response.json()

# 설정 확인
if not MCP_BASE_URL or not subscription_key:
    print("❌ MCP 서버 설정이 필요합니다.")
    print("   1. apim-mcp.ipynb 실행 완료")
    print("   2. 섹션 6.4 실행하여 workshop_config.json 저장")
    print("   3. 이 노트북의 1.3 셀 다시 실행")
else:
    # MCP 클라이언트 인스턴스 생성
    mcp_client = MCPClient(MCP_BASE_URL, subscription_key, JWT_TOKEN)
    
    print("✅ MCP 클라이언트 생성 완료")
    print()
    print("🔐 인증 설정 상태:")
    print("=" * 60)
    print(f"   📋 subscription_key: ✅ 설정됨")
    print(f"   🎫 JWT_TOKEN: {'✅ 사용' if JWT_TOKEN else '⚪ 미사용'}")
    print()
    if JWT_TOKEN:
        print("   ✅ JWT 인증이 활성화되었습니다.")
        print("      Authorization: Bearer <token>")
    else:
        print("💡 JWT 토큰 미사용 시:")
        print("   ├─ apim-mcp.ipynb에서 validate-jwt 정책이 적용된 경우")
        print("   │   → 401 Unauthorized 발생")
        print("   │")
        print("   └─ JWT 토큰 획득 방법:")
        print("       apim-mcp.ipynb 섹션 4.1에서 Entra ID 앱 생성 후")
        print("       섹션 6.4 실행하여 설정 저장")
    print("=" * 60)

### 2.2 연결 테스트 실행

MCP 클라이언트를 사용하여 서버 연결을 테스트합니다.

**테스트 순서:**
1. **Health Check**: 서버가 정상 동작하는지 확인
2. **List Tools**: 사용 가능한 도구 목록 조회
3. **Call Tool**: `echo` 도구로 실제 실행 테스트

> ⚠️ 연결 실패 시 확인사항:
> - APIM Gateway URL이 올바른지 확인
> - Subscription Key가 유효한지 확인
> - MCP 서버가 배포되었는지 확인 (`apim-mcp.ipynb` 실행)

In [ ]:
# MCP 서버 연결 테스트
print("🔍 MCP 서버 연결 테스트")
print("=" * 60)

try:
    # Health Check
    print("\n1️⃣ Health Check:")
    health = mcp_client.health_check()
    print(f"   ✅ 상태: {health.get('status')}")
    print(f"   📦 서버: {health.get('server')}")
    print(f"   🕐 시간: {health.get('timestamp')}")
    
    # List Tools
    print("\n2️⃣ 사용 가능한 도구:")
    tools = mcp_client.list_tools()
    for tool in tools:
        print(f"   • {tool['name']}: {tool['description']}")
    
    # Tool Test
    print("\n3️⃣ 도구 테스트 (echo):")
    result = mcp_client.call_tool("echo", {"message": "Hello from LLM test!"})
    print(f"   ✅ 결과: {result['content'][0]['text']}")
    
    print("\n" + "=" * 60)
    print("✅ MCP 서버 연결 성공!")
    
except Exception as e:
    print(f"\n❌ MCP 서버 연결 실패: {str(e)}")
    print("\n💡 확인사항:")
    print("   1. APIM_GATEWAY_URL이 올바른지 확인")
    print("   2. APIM_SUBSCRIPTION_KEY가 설정되었는지 확인")
    print("   3. MCP 서버가 배포되었는지 확인 (apim-mcp.ipynb 실행)")

---
<a id='3'></a>
# 3️⃣ Azure OpenAI와 MCP 연동

Azure OpenAI의 **Function Calling** 기능을 사용하여 MCP 도구를 LLM과 연동합니다.

## Function Calling 동작 원리

```
1. 사용자 → LLM: "지금 몇 시야?"
2. LLM → 클라이언트: tool_calls: [{name: "get_current_time", arguments: {...}}]
3. 클라이언트 → MCP: POST /mcp/tools {"name": "get_current_time", ...}
4. MCP → 클라이언트: {"content": [{"text": "2024-12-09 15:30:00 KST"}]}
5. 클라이언트 → LLM: tool result: "2024-12-09 15:30:00 KST"
6. LLM → 사용자: "현재 한국 시간은 오후 3시 30분입니다."
```

## MCP → OpenAI 도구 형식 변환

| MCP 스키마 | OpenAI Function 형식 |
|------------|---------------------|
| `name` | `function.name` |
| `description` | `function.description` |
| `inputSchema` | `function.parameters` |

### 3.1 도구 형식 변환

MCP 도구 스키마를 OpenAI Function Calling 형식으로 변환하는 함수를 정의합니다.

**변환 예시:**

```json
// MCP 형식
{
  "name": "calculate",
  "description": "수학 계산을 수행합니다",
  "inputSchema": {
    "type": "object",
    "properties": {
      "operation": {"type": "string"},
      "a": {"type": "number"},
      "b": {"type": "number"}
    }
  }
}

// OpenAI Function 형식
{
  "type": "function",
  "function": {
    "name": "calculate",
    "description": "수학 계산을 수행합니다",
    "parameters": {...}
  }
}
```

In [ ]:
# MCP 도구를 OpenAI Function 형식으로 변환
def mcp_tools_to_openai_functions(mcp_tools: list) -> list:
    """
    MCP 도구 스키마를 OpenAI Function 형식으로 변환
    
    MCP 형식:
    {
        "name": "calculate",
        "description": "...",
        "inputSchema": { "type": "object", "properties": {...} }
    }
    
    OpenAI 형식:
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "...",
            "parameters": { "type": "object", "properties": {...} }
        }
    }
    """
    openai_tools = []
    for tool in mcp_tools:
        openai_tool = {
            "type": "function",
            "function": {
                "name": tool["name"],
                "description": tool["description"],
                "parameters": tool.get("inputSchema", {"type": "object", "properties": {}})
            }
        }
        openai_tools.append(openai_tool)
    return openai_tools

# MCP 도구를 OpenAI Function 형식으로 변환
mcp_tools = mcp_client.list_tools()
openai_tools = mcp_tools_to_openai_functions(mcp_tools)

print("📋 OpenAI Function 형식으로 변환된 도구:")
for tool in openai_tools:
    print(f"   • {tool['function']['name']}")
print()
print(json.dumps(openai_tools, indent=2, ensure_ascii=False))

### 3.2 Azure OpenAI 클라이언트 생성

Azure OpenAI 클라이언트를 생성합니다.

| 항목 | 설명 |
|------|------|
| **클라이언트** | `AzureOpenAI` |
| **모델 지정** | 배포 이름 (Deployment Name) |
| **인증** | API 키 또는 Entra ID |

In [ ]:
# Azure OpenAI 클라이언트 생성
llm_client = AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION
)
model_name = AZURE_OPENAI_DEPLOYMENT
print(f"✅ Azure OpenAI 클라이언트 생성 (배포: {model_name})")

### 3.3 MCP 에이전트 함수

사용자 메시지를 받아 LLM과 MCP 서버를 연동하여 응답을 생성하는 에이전트 함수입니다.

**실행 흐름:**

```
┌──────────────────────────────────────────────────────────────┐
│  1. 사용자 메시지 → LLM 전송 (tools 포함)                    │
├──────────────────────────────────────────────────────────────┤
│  2. LLM 응답 확인                                            │
│     ├─ tool_calls 있음 → MCP 서버에서 도구 실행             │
│     │                  → 결과를 LLM에 전송                   │
│     │                  → 최종 응답 생성                      │
│     └─ tool_calls 없음 → 직접 응답                          │
├──────────────────────────────────────────────────────────────┤
│  3. 최종 응답 반환                                           │
└──────────────────────────────────────────────────────────────┘
```

**주요 파라미터:**
- `user_message`: 사용자 입력 메시지
- `verbose`: 디버그 출력 여부 (기본값: True)

In [ ]:
def run_mcp_agent(user_message: str, verbose: bool = True) -> str:
    """
    MCP 도구를 사용하는 LLM 에이전트 실행
    
    1. 사용자 메시지를 LLM에 전송
    2. LLM이 도구 호출을 요청하면 MCP 서버에서 실행
    3. 결과를 LLM에 반환하여 최종 응답 생성
    """
    messages = [
        {
            "role": "system",
            "content": """당신은 MCP (Model Context Protocol) 도구를 사용할 수 있는 AI 어시스턴트입니다.
사용 가능한 도구:
- echo: 메시지를 그대로 반환 (테스트용)
- get_current_time: 현재 시간 조회 (UTC 또는 KST)
- calculate: 사칙연산 수행 (add, subtract, multiply, divide)

사용자의 요청에 따라 적절한 도구를 사용하여 답변하세요."""
        },
        {"role": "user", "content": user_message}
    ]
    
    if verbose:
        print(f"👤 사용자: {user_message}")
        print()
    
    # 1. LLM 호출 (도구 사용 가능)
    response = llm_client.chat.completions.create(
        model=model_name,
        messages=messages,
        tools=openai_tools,
        tool_choice="auto"
    )
    
    assistant_message = response.choices[0].message
    
    # 2. 도구 호출이 필요한 경우
    if assistant_message.tool_calls:
        messages.append(assistant_message)
        
        for tool_call in assistant_message.tool_calls:
            tool_name = tool_call.function.name
            tool_args = json.loads(tool_call.function.arguments)
            
            if verbose:
                print(f"🔧 도구 호출: {tool_name}")
                print(f"   인자: {json.dumps(tool_args, ensure_ascii=False)}")
            
            # MCP 서버에서 도구 실행
            try:
                result = mcp_client.call_tool(tool_name, tool_args)
                tool_result = result["content"][0]["text"]
                
                if verbose:
                    print(f"   결과: {tool_result}")
                    print()
            except Exception as e:
                tool_result = f"오류: {str(e)}"
                if verbose:
                    print(f"   ❌ 오류: {str(e)}")
                    print()
            
            # 도구 결과를 메시지에 추가
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": tool_result
            })
        
        # 3. 도구 결과를 포함하여 LLM 재호출
        final_response = llm_client.chat.completions.create(
            model=model_name,
            messages=messages
        )
        final_answer = final_response.choices[0].message.content
    else:
        # 도구 호출 없이 직접 응답
        final_answer = assistant_message.content
    
    if verbose:
        print(f"🤖 AI: {final_answer}")
    
    return final_answer

### 3.4 MCP 에이전트 테스트

다양한 시나리오로 MCP 에이전트를 테스트합니다.

| 테스트 | 설명 | 예상 도구 호출 |
|--------|------|---------------|
| **현재 시간** | 시간 조회 요청 | `get_current_time` |
| **계산** | 수학 연산 요청 | `calculate` |
| **복합 요청** | 여러 작업 동시 요청 | 다중 도구 호출 |
| **Echo** | 메시지 반복 요청 | `echo` |

In [ ]:
# 테스트 1: 현재 시간 조회
print("=" * 60)
print("테스트 1: 현재 시간 조회")
print("=" * 60)
run_mcp_agent("지금 한국 시간이 몇 시야?")

In [ ]:
# 테스트 2: 계산
print("=" * 60)
print("테스트 2: 계산")
print("=" * 60)
run_mcp_agent("123 곱하기 456은 얼마야?")

In [ ]:
# 테스트 3: 복합 요청
print("=" * 60)
print("테스트 3: 복합 요청")
print("=" * 60)
run_mcp_agent("현재 시간을 알려주고, 25 나누기 5의 결과도 알려줘")

In [ ]:
# 테스트 4: Echo 테스트
print("=" * 60)
print("테스트 4: Echo 테스트")
print("=" * 60)
run_mcp_agent("'안녕하세요, MCP 서버!'라는 메시지를 echo 해줘")

---
<a id='4'></a>
# 4️⃣ 대화형 MCP 에이전트

대화 기록을 유지하면서 MCP 도구를 사용하는 에이전트입니다.

## 대화형 에이전트의 특징

| 기능 | 설명 |
|------|------|
| **대화 기록 유지** | 이전 대화 내용을 기억하고 문맥 유지 |
| **도구 호출 투명성** | 어떤 도구가 호출되었는지 실시간 표시 |
| **세션 관리** | `reset()` 메서드로 대화 초기화 가능 |

## 단발성 에이전트 vs 대화형 에이전트

```
# 단발성 (run_mcp_agent)
사용자: "10 곱하기 20은?"  →  AI: "200입니다"
사용자: "그 결과에 5를 더하면?"  →  AI: "무엇의 결과인가요?" (❌ 문맥 없음)

# 대화형 (MCPChatAgent)
사용자: "10 곱하기 20은?"  →  AI: "200입니다"
사용자: "그 결과에 5를 더하면?"  →  AI: "205입니다" (✅ 문맥 유지)
```

### 4.1 MCPChatAgent 클래스

대화 기록을 유지하는 MCP 에이전트 클래스입니다.

**클래스 구조:**

```python
class MCPChatAgent:
    ├── __init__(mcp_client, llm_client, model_name)  # 초기화
    │   └── messages: list  # 대화 기록 저장
    ├── chat(user_message) -> str                     # 대화 수행
    │   ├── 메시지 추가
    │   ├── LLM 호출
    │   ├── 도구 호출 처리
    │   └── 응답 반환
    └── reset()                                       # 대화 기록 초기화
```

**메시지 구조 (OpenAI Chat API):**
```python
messages = [
    {"role": "system", "content": "시스템 프롬프트"},
    {"role": "user", "content": "사용자 메시지"},
    {"role": "assistant", "content": "AI 응답", "tool_calls": [...]},
    {"role": "tool", "tool_call_id": "...", "content": "도구 결과"},
    ...
]
```

In [ ]:
class MCPChatAgent:
    """대화 기록을 유지하는 MCP 에이전트"""
    
    def __init__(self, mcp_client: MCPClient, llm_client, model_name: str):
        self.mcp_client = mcp_client
        self.llm_client = llm_client
        self.model_name = model_name
        self.tools = mcp_tools_to_openai_functions(mcp_client.list_tools())
        self.messages = [
            {
                "role": "system",
                "content": """당신은 MCP 도구를 사용할 수 있는 AI 어시스턴트입니다.
사용 가능한 도구: echo, get_current_time, calculate
친절하고 도움이 되는 방식으로 대화하세요."""
            }
        ]
    
    def chat(self, user_message: str) -> str:
        """대화 수행"""
        self.messages.append({"role": "user", "content": user_message})
        
        # LLM 호출
        response = self.llm_client.chat.completions.create(
            model=self.model_name,
            messages=self.messages,
            tools=self.tools,
            tool_choice="auto"
        )
        
        assistant_message = response.choices[0].message
        
        # 도구 호출 처리
        if assistant_message.tool_calls:
            self.messages.append(assistant_message)
            
            for tool_call in assistant_message.tool_calls:
                tool_name = tool_call.function.name
                tool_args = json.loads(tool_call.function.arguments)
                
                print(f"   🔧 {tool_name}({tool_args})")
                
                try:
                    result = self.mcp_client.call_tool(tool_name, tool_args)
                    tool_result = result["content"][0]["text"]
                except Exception as e:
                    tool_result = f"오류: {str(e)}"
                
                self.messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": tool_result
                })
            
            # 최종 응답
            final_response = self.llm_client.chat.completions.create(
                model=self.model_name,
                messages=self.messages
            )
            final_answer = final_response.choices[0].message.content
        else:
            final_answer = assistant_message.content
        
        self.messages.append({"role": "assistant", "content": final_answer})
        return final_answer
    
    def reset(self):
        """대화 기록 초기화"""
        self.messages = [self.messages[0]]  # 시스템 메시지만 유지

# 에이전트 인스턴스 생성
agent = MCPChatAgent(mcp_client, llm_client, model_name)
print("✅ MCP Chat Agent 생성 완료")

### 4.2 대화형 테스트

연속된 대화로 에이전트가 문맥을 유지하는지 테스트합니다.

**테스트 시나리오:**
1. 현재 시간 조회
2. 계산 수행 (10 × 20)
3. 이전 결과 참조 (결과 + 5) → 문맥 유지 확인

In [ ]:
# 대화형 테스트
print("=" * 60)
print("🗣️ 대화형 MCP 에이전트 테스트")
print("=" * 60)

# 대화 1
print("\n👤 사용자: 안녕! 지금 몇 시야?")
response = agent.chat("안녕! 지금 몇 시야?")
print(f"🤖 AI: {response}")

# 대화 2 (이전 대화 참조)
print("\n👤 사용자: 10 곱하기 20은?")
response = agent.chat("10 곱하기 20은?")
print(f"🤖 AI: {response}")

# 대화 3
print("\n👤 사용자: 방금 계산 결과에 5를 더하면?")
response = agent.chat("방금 계산 결과에 5를 더하면?")
print(f"🤖 AI: {response}")

### 4.3 인터랙티브 대화 모드

실시간으로 대화할 수 있는 인터랙티브 모드입니다.

**명령어:**
| 입력 | 동작 |
|------|------|
| 일반 텍스트 | AI와 대화 |
| `reset` | 대화 기록 초기화 |
| `quit` / `exit` / `q` | 대화 종료 |

> 💡 **Tip**: Jupyter 노트북에서 실행 시 셀을 중지하면 대화가 종료됩니다.

In [ ]:
# 인터랙티브 대화 모드
print("=" * 60)
print("🗣️ 인터랙티브 대화 모드")
print("   'quit' 또는 'exit'를 입력하면 종료됩니다.")
print("   'reset'을 입력하면 대화 기록을 초기화합니다.")
print("=" * 60)

agent.reset()

while True:
    try:
        user_input = input("\n👤 You: ").strip()
        
        if not user_input:
            continue
        
        if user_input.lower() in ['quit', 'exit', 'q']:
            print("👋 대화를 종료합니다.")
            break
        
        if user_input.lower() == 'reset':
            agent.reset()
            print("🔄 대화 기록이 초기화되었습니다.")
            continue
        
        response = agent.chat(user_input)
        print(f"\n🤖 AI: {response}")
        
    except KeyboardInterrupt:
        print("\n👋 대화를 종료합니다.")
        break
    except Exception as e:
        print(f"\n❌ 오류: {str(e)}")

---
# 🎉 완료

## ✅ 학습 완료 항목

- [x] MCP 클라이언트 구현 (Python `requests` 기반)
- [x] MCP → OpenAI Function 형식 변환
- [x] Azure OpenAI Function Calling 연동
- [x] 단발성 에이전트 (`run_mcp_agent`)
- [x] 대화형 에이전트 (`MCPChatAgent`)
- [x] 인터랙티브 대화 모드

## 🚀 다음 단계

| 과제 | 설명 |
|------|------|
| **커스텀 도구 추가** | 데이터베이스 조회, 외부 API 호출 등 |
| **스트리밍 응답** | SSE/WebSocket으로 실시간 응답 |
| **멀티모달 지원** | 이미지 분석, 파일 처리 도구 |
| **프로덕션 배포** | 웹 앱, Slack 봇, Teams 통합 |

---

**📚 관련 리소스:**
- [Azure OpenAI Function Calling](https://learn.microsoft.com/azure/ai-services/openai/how-to/function-calling)
- [MCP Protocol Specification](https://modelcontextprotocol.io/)